# Conceptual Qk-NN (Quantum Distance Estimation)

This notebook demonstrates a simplified implementation of the *quantum distance estimation* part of Quantum k-Nearest Neighbors (Qk-NN) using Qiskit. It focuses on the **Swap Test**, which is used to estimate the distance between two vectors represented as quantum states.  This is *not* a complete Qk-NN implementation, but it illustrates the key quantum principle.

**We will:**

1.  Encode vectors using a simplified amplitude encoding.
2.  Implement the Swap Test circuit.
3.  Estimate distances using the Swap Test.
4.  Show a basic example of comparing distances.

---


## Step 1: Import Necessary Libraries

We begin by importing the necessary libraries:

*   **Qiskit:**
    *   `QuantumCircuit`: For building quantum circuits.
    *   `QuantumRegister`: For creating groups of qubits.
    *   `ClassicalRegister`: For storing measurement results (classical bits).
    *   `transpile`: For optimizing circuits for a specific backend (simulator or hardware).
*   **Qiskit Aer:**
    *   `AerSimulator`: For simulating quantum circuits on a classical computer.
*   **NumPy:**
    *   `import numpy as np`: For numerical operations (arrays, mathematical functions), using the alias `np`.

In [7]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
import numpy as np

## Step 2: Define the Encoding Function (`encode_vector`)

The `encode_vector` function performs a simplified *amplitude encoding*:

*   **Function Definition:** `def encode_vector(vector):`
    *   `vector`: A *normalized* 2D NumPy array (e.g., `[0.8, 0.6]`).

*   **Docstring:**  Explains the function's purpose (amplitude encoding).

*   **Simplification Note:**  This is a simplified encoding *only for 2D vectors*. Real-world amplitude encoding is much more complex.

*   **Quantum Circuit Creation:** `qc = QuantumCircuit(1)`: Creates a `QuantumCircuit` with *one* qubit (sufficient for a 2D vector).

*   **Rotation Gate:** `qc.ry(2 * np.arccos(vector[0]), 0)`: Applies an `ry` gate (Y-axis rotation) to the qubit.
    *   `np.arccos(vector[0])`: Calculates the arccosine of the vector's first element (to get the angle).
    *   `2 * ...`:  Multiplies the angle by 2. The `ry` gate's angle parameter is *half* the rotation angle on the Bloch sphere.
    *   `0`: Applies the gate to qubit 0.

*   **Return Value:** `return qc`: Returns the `QuantumCircuit` representing the encoded state.

**In summary, this function takes a normalized 2D vector and creates a quantum circuit that prepares a single qubit in a state whose amplitudes correspond to the vector's components.**


In [8]:
def encode_vector(vector):
    """Encodes a normalized vector into a quantum state (amplitude encoding)."""
    # VERY simplified for 2D vectors only.  Real-world encoding is more complex.
    qc = QuantumCircuit(1)
    qc.ry(2 * np.arccos(vector[0]), 0) # Encode angle based on vector components
    return qc

## Step 3: Define the Swap Test Function (`swap_test`)

The `swap_test` function implements the quantum Swap Test:

*   **Function Definition:** `def swap_test(vector1, vector2):`
    *   `vector1`: The first normalized 2D vector.
    *   `vector2`: The second normalized 2D vector.

*   **Encoding:**
    *   `encoded_vec1 = encode_vector(vector1)`: Encodes `vector1`.
    *   `encoded_vec2 = encode_vector(vector2)`: Encodes `vector2`.

*   **Register Creation:**
    *   `qreg_q = QuantumRegister(3, 'q')`: Creates a quantum register with *three* qubits ('q').
    *   `creg_c = ClassicalRegister(1, 'c')`: Creates a classical register with *one* bit ('c').

*   **Circuit Creation:** `circuit = QuantumCircuit(qreg_q, creg_c)`: Creates a `QuantumCircuit`.

*   **Composing Encoded Vectors:**
    *   `circuit.compose(encoded_vec1, qubits=[1], inplace=True)`: Adds `encoded_vec1` to qubit 1.
    *   `circuit.compose(encoded_vec2, qubits=[2], inplace=True)`: Adds `encoded_vec2` to qubit 2.

*   **Hadamard Gate:** `circuit.h(qreg_q[0])`: Applies a Hadamard gate to the control qubit (qubit 0).

*   **CSWAP Gate:** `circuit.cswap(qreg_q[0], qreg_q[1], qreg_q[2])`: Applies the controlled-SWAP.
    *   Control qubit: `qreg_q[0]`
    *   Target qubits: `qreg_q[1]` and `qreg_q[2]`

*   **Second Hadamard Gate:** `circuit.h(qreg_q[0])`: Applies another Hadamard to the control qubit.

*   **Measurement:** `circuit.measure(qreg_q[0], creg_c[0])`: Measures the control qubit.

*   **Return Value:** `return circuit`: Returns the Swap Test circuit.

**This function constructs the Swap Test circuit, the core quantum component for estimating the distance.**


In [9]:
def swap_test(vector1, vector2):
    """Performs a Swap Test to estimate distance between two vectors."""

    encoded_vec1 = encode_vector(vector1)
    encoded_vec2 = encode_vector(vector2)

    # Create quantum and classical registers
    qreg_q = QuantumRegister(3, 'q')
    creg_c = ClassicalRegister(1, 'c')
    circuit = QuantumCircuit(qreg_q, creg_c)

    # Initialize qubits with encoded vectors
    circuit.compose(encoded_vec1, qubits=[1], inplace=True)
    circuit.compose(encoded_vec2, qubits=[2], inplace=True)

    # Apply Hadamard to the control qubit
    circuit.h(qreg_q[0])
    # Apply CSWAP (controlled-SWAP) gate
    circuit.cswap(qreg_q[0], qreg_q[1], qreg_q[2])
    # Apply Hadamard to the control qubit
    circuit.h(qreg_q[0])
    # Measure the control qubit
    circuit.measure(qreg_q[0], creg_c[0])

    return circuit

## Step 4: Define the Distance Estimation Function (`estimate_distance`)

The `estimate_distance` function estimates the distance between two vectors:

*   **Function Definition:** `def estimate_distance(vector1, vector2, shots=1024):`
    *   `vector1`, `vector2`: The input vectors.
    *   `shots=1024`: The number of simulation runs (default: 1024).

*   **Circuit Creation:** `circuit = swap_test(vector1, vector2)`: Creates the Swap Test circuit.

*   **Simulation Setup:**
    *   `simulator = AerSimulator()`: Creates an `AerSimulator`.
    *   `compiled_circuit = transpile(circuit, simulator)`: Optimizes the circuit.
    *   `job = simulator.run(compiled_circuit, shots=shots)`: Runs the simulation.
    *   `result = job.result()`: Gets the results.
    *   `counts = result.get_counts(circuit)`: Gets measurement counts.

*   **Probability Calculation:** `prob_0 = counts.get('0', 0) / shots`: Calculates P(0).

*   **Distance Calculation:**
    *   `distance_squared = 2 * (1 - prob_0)`: Calculates squared distance.
    *   `distance = np.sqrt(max(0, distance_squared))`: Calculates distance.

*   **Return Value:** `return distance`: Returns the estimated distance.

**This function orchestrates the simulation of the Swap Test and extracts the estimated distance.**


In [10]:
def estimate_distance(vector1, vector2, shots=1024):
    """Estimates distance using the Swap Test results."""

    circuit = swap_test(vector1, vector2)

    # Simulate the circuit
    simulator = AerSimulator()
    compiled_circuit = transpile(circuit, simulator)
    job = simulator.run(compiled_circuit, shots=shots)
    result = job.result()
    counts = result.get_counts(circuit)

    # Calculate probability of measuring 0
    prob_0 = counts.get('0', 0) / shots

    # Distance estimation (derived from Swap Test probability)
    distance_squared = 2 * (1 - prob_0)  # 1 - <psi1|psi2>  (inner product)
    distance = np.sqrt(max(0, distance_squared))  # Ensure non-negative

    return distance

## Step 5: Example Usage

This section demonstrates how to *use* the defined functions:

*   **Data Definition:**
    *   `data_point1 = ...`, `data_point2 = ...`, `new_point = ...`: Defines example data points.

*   **Distance Calculation:**
    *   `distance1 = ...`: Calculates distance between `new_point` and `data_point1`.
    *   `distance2 = ...`: Calculates distance between `new_point` and `data_point2`.

*   **Output:** `print(...)`: Prints the estimated distances.

*   **Comparison:** `if ... else ...`: Compares distances.

*   **Important Note:** This is a simplified example. A complete Qk-NN would require calculating distances to all training points and finding the *k* smallest distances, along with a final classical majority vote.


In [14]:
# --- Example Usage ---
# Two very simple 2D data points (normalized)
data_point1 = np.array([0.8, 0.6])
data_point2 = np.array([0.6, 0.8])
new_point = np.array([0.7, 0.71])  # close to equal distance

# Estimate distances using the quantum Swap Test
distance1 = estimate_distance(new_point, data_point1)
distance2 = estimate_distance(new_point, data_point2)

print(f"Estimated distance to point 1: {distance1:.4f}")
print(f"Estimated distance to point 2: {distance2:.4f}")

# Find the smallest distance (this would be repeated for all data points in a full Qk-NN)
#  In a real implementation, you'd use a quantum algorithm to find the k-smallest.
if distance1 < distance2:
    print("Point 1 is closer.")
else:
    print("Point 2 is closer.")

Estimated distance to point 1: 0.1654
Estimated distance to point 2: 0.1169
Point 2 is closer.
